# Exploring ALS and TLS Point Clouds

At the conclusion of this module, you will be able to:

1. Discover how ALS and TLS datasets are captured and their real-world applications in forestry.
2. Identify the structure of ALS and TLS datasets, including key attributes like coordinates, intensity, and color (RGB).
3. Create visualizations and perform basic machine learning tasks using ALS and TLS datasets.

## ALS Data

Airborne Laser Scanning (ALS) is a LiDAR technique that uses sensors mounted on aircraft or drones to collect data over large areas. ALS is widely used in applications such as forestry, topographic mapping and urban planning due to its ability to capture high-resolution 3D data over large landscapes. The data collected typically includes coordinates (X, Y, Z), intensity (reflection of the laser pulse) and sometimes RGB values. ALS is particularly effective for analysing terrain, canopy structure and vegetation density because it can penetrate through gaps in the canopy to produce detailed elevation models.

## Our Data
The data we’re working with is in the LAZ file format, which is a compressed version of the LAS format. If you want to dive deeper into this format, check out the [**American Society for Photogrammetry and Remote Sensing (ASPRS)** site](https://www.asprs.org/divisions-committees/lidar-division/laser-las-file-format-exchange-activities). 

Each file is about 150MB, and since the data is already hosted in an open cloud, downloading it to our server isn’t the most efficient option. Instead, we’ll load the data directly into memory for processing.

To work with this data, we'll use the [laspy](https://laspy.readthedocs.io/en/latest/index.html) library, a powerful tool for handling LAS and LAZ files.

In [ ]:
import requests
import laspy
from io import BytesIO

# 1. We set the URL for ALS file
als_url = "https://wifire-data.sdsc.edu/nc/s/nnSMqWfZAN6Cz6m/download?path=%2Fals_data&files=USGS_LPC_CA_SierraNevada_B22_10SGJ3270.laz"

# 2. We fetch ALS file content
response_als = requests.get(als_url)

# 3. We load the ALS data into memory
als_buffer = BytesIO(response_als.content)

# 4. We open the file using laspy, and print the format and number of points.
with laspy.open(als_buffer) as als_file:
    print(f"ALS Point Format: {als_file.header.point_format}")
    print(f"ALS Number of Points: {als_file.header.point_count}")

# 5. We also open the file for next tasks
    als_points = als_file.read()

The first thing to note is that our data uses Point Record Format 6. If you're curious about the different point record formats, you can check out [this document](https://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf) (pages 15-36).

Another interesting detail is that our dataset contains almost 23 million points! That’s a massive amount of data, right?

Next, let’s double-check the format of our data by examining the dimensions available for each point.

In [ ]:
# We need to convert the object into a list for proper display
list(als_points.point_format.dimension_names)

As we see, the information corresponds to the expected columns given the data format. Now, let's extract x, y and z and take a look at the first 5 points 

In [ ]:
# Save XYZ as np arrays and print the first 5 points consecutively. 
x_als, y_als, z_als = als_points.x, als_points.y, als_points.z
print(f"First 5 ALS points (x, y, z): {list(zip(x_als[:5], y_als[:5], z_als[:5]))}")

## Visualizing the data

Now that we have our data loaded into memory, let’s create a simple 3D visualization using [Plotly](https://plotly.com/python/). 

For this example, we’ll work with a sample of **50,000 points**. Visualizing the entire dataset would be too heavy for Jupyter, but this sample will give us a good idea of what the data looks like.

If you’re curious to explore the full dataset, you can use dedicated software like [Cloud Compare](https://www.danielgm.net/cc/), which is specifically designed for handling large point clouds..  

In [ ]:
import plotly.graph_objects as go

subset_size = 50000  # You can change this value, but this sample should be enough for our visualization purposes.

# 2. We create an interactive 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x_als[:subset_size],
    y=y_als[:subset_size],
    z=z_als[:subset_size],
    mode='markers',
    marker=dict(
        size=2,                     
        color=z_als[:subset_size],   # Color points by height (just to make it look nicer :D)
        colorscale='Viridis',        
        opacity=0.8                  
    )
)])

# 3. We add a few details and show the figure
fig.update_layout(
    title="Airborne Laser Scanning (ALS) Point Cloud",
    scene=dict(
        xaxis_title="X (m)",
        yaxis_title="Y (m)",
        zaxis_title="Z (m)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)  
)

fig.show()

Looks cool, right? As you can see, the visualization resembles the shape of a forest. If you zoom in, you’ll notice that each tree is made up of thousands of individual points, capturing fine details of the structure.

## TLS Data

Terrestrial Laser Scanning (TLS) is a ground-based LiDAR technique where sensors are stationed on tripods or vehicles to capture highly detailed 3D data of localized areas. Unlike ALS, which covers broad landscapes, TLS excels at capturing intricate structures, such as tree trunks, midstory vegetation, or buildings, with exceptional precision. TLS datasets often include coordinates (X, Y, Z), intensity values, and sometimes RGB colors, enabling detailed analysis of structural features. This technology is widely used in forestry for measuring tree dimensions, in architecture for building modeling, and in geology for mapping rock formations. 

## Our data

The TLS files we’re working with are in the **Point Cloud Data Format (.ptx)**, and they’re quite large—each file is around 500MB. Downloading the entire set would be very inefficient, and processing files in this format directly with Python can be quite challenging.

Thankfully, we have a tool called [PDAL](https://pdal.io/en/2.8.2/) that can help! PDAL allows us to convert our .ptx files into LAZ files without losing any information. Here’s the plan:

1. Temporarily download the `.ptx` file to our server.
2. Use PDAL to translate the `.ptx` file into a `.laz` file. (This is a terminal process, so we’ll need to use Python’s [subprocess](https://docs.python.org/3/library/subprocess.html) library.)
3. Once the conversion is done, delete the original `.ptx` file to save space.

In [ ]:
import requests
from pathlib import Path
import subprocess

# URL of the PTX file
ptx_url = "https://wifire-data.sdsc.edu/nc/s/5oiFgzCdo4QPi34/download?path=%2FLidar_data&files=CATNF_6041_20240814_1.ptx"

# Filenames - You can replace these names
ptx_file = Path("input.ptx")
laz_file = Path("output.laz")

# 1. We request and download the .ptx file locally
response = requests.get(ptx_url)

with ptx_file.open("wb") as f:
    f.write(response.content)

print(f"Downloaded PTX file: {ptx_file}")

# 2. We convert the .ptx file to .laz using PDAL
pdal_command = ["pdal", "translate", str(ptx_file), str(laz_file)]
result = subprocess.run(pdal_command, capture_output=True, text=True)

# This is a check in case there is some error with the transformation
if result.returncode == 0:
    print(f"Converted {ptx_file} to {laz_file}")
else:
    print(f"PDAL translation failed: {result.stderr}")
    raise RuntimeError(f"PDAL translation error: {result.stderr}")

# 3. We delete the original .ptx file
ptx_file.unlink()
print(f"Deleted original PTX file: {ptx_file}")

Success! 🎉 If you check the size of the `output.laz` file, you’ll notice it’s only around 25–30MB—much smaller than the original file. While we’ll delete it in the next cell, this compressed format will definitely make our lives easier as we move on to the next steps.

Now, let's "recycle" our previous code to load the data using laspy.

In [ ]:
# Read the .las file using Laspy
with laspy.open(laz_file) as tls_file:
    print(f"Point Format: {tls_file.header.point_format}")
    print(f"Number of Points: {tls_file.header.point_count}")

    # Read all points from the LAZ file
    tls_points = tls_file.read()
    list(tls_points.point_format.dimension_names)

# Now we can remove the .laz file
laz_file.unlink()
print(f"Deleted original LAZ file: {laz_file}")

The first thing we notice is that the format of this data is different from the previous one. Another key difference is that this file contains fewer points. Let’s take a closer look at the columns in this file to understand its structure.

In [ ]:
# We recycle code again :)
list(tls_points.point_format.dimension_names)

One key difference between this dataset and the previous one is that it includes Red, Green, and Blue (RGB) colors. This additional information can be incredibly useful, especially for classification tasks, as it helps distinguish between different objects more effectively.

Let's take a look at the first 5 data points of RGB.

In [ ]:
r_tls, g_tls, b_tls = tls_points.red, tls_points.green, tls_points.blue
print(f"\nFirst 5 points (R, G, B): {list(zip(r_tls[:5], g_tls[:5], b_tls[:5]))}")

Note: Since .ptx is a human-readable format, you can alternatively parse these files directly and load the data into an array using NumPy. 

In [ ]:
import numpy as np
with ptx_file.open("wb") as f:
    f.write(response.content)

n_header = 10 # Number of rows in the header

tls_ptx = np.loadtxt('input.ptx', skiprows=n_header)
tls_ptx = tls_ptx[~np.all(tls_ptx == 0, axis=1)]
print(tls_ptx.shape)

ptx_file.unlink()

## Visualizing the data

Just like we did with the ALS data, we’ll create a quick visualization of this TLS data. Keep in mind that since this data was captured from the ground, it’ll be harder to recognize the overall shape of the point clouds.

For a more detailed and comprehensive visualization of this data, feel free to check out [this site](https://burnpro3d.sdsc.edu/points2pano/?plot=CATNF_6022&ts=20240731). The site was developed by the [WIFIRE](https://wifire.ucsd.edu/) team and uses TLS data to simulate a forest experience.

In [ ]:
import numpy as np

subset_size = 100000  # Number of points to sample

# We generate a set of random points
random_indices = np.random.choice(len(tls_points.x), size=subset_size, replace=False)

# Use the random indices to select points
fig = go.Figure(data=[go.Scatter3d(
    x=tls_points.x[random_indices],
    y=tls_points.y[random_indices],
    z=tls_points.z[random_indices],
    mode='markers',
    marker=dict(
        size=2,
        color=r_tls[random_indices],  # Let's use red as the scale for our colors
        colorscale='Reds',
        opacity=0.8
    )
)])

fig.update_layout(
    title="3D Point Cloud Visualization (LAZ Data)",
    scene=dict(
        xaxis_title="X (m)",
        yaxis_title="Y (m)",
        zaxis_title="Z (m)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)  
)

# Display the plot
fig.show()


As we can observe, this visualization isn’t as easy to interpret as the earlier one. But if you look closely, you can start to spot the shape of the forest from the ground level, with the points coming together to form the tree trunks.

## A simple application

The ALS files we are working with include a column called classification, which provides a label for each point in the dataset. These labels represent the category of each point, such as ground, vegetation, building, and others. This classification is critical for interpreting point cloud data and is often used in applications like terrain mapping, vegetation analysis, and urban planning.

The classification process is typically performed using a combination of manual annotation by experts and automated methods involving sophisticated Machine Learning models. Pages 17-19 of [this document](https://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf) provide detailed information about the classification standards and their specific meanings.

For this demo, we will take a sample of points and use it to build a simple point cloud classifier. Our goal is to leverage the available data to train a model capable of predicting the class of a point based on its attributes. 

As you might expect, processing and working with ALS and TLS data presents significant challenges due to the sheer size of the datasets. To handle this effectively, we need to leverage specialized libraries and resources. For this module, we’ve chosen Dask to process the data and train a simple machine learning model.

Why Dask? Dask is designed for parallel and distributed computing, making it an excellent choice for large-scale data processing. It efficiently utilizes all available CPU cores by splitting data into manageable chunks and processing them in parallel.

In the instructions for this module, you were asked to reserve 2 cores. Let’s start by confirming the availability of these resources.

In [ ]:
from dask.system import cpu_count

cores = cpu_count()
print(f"Number of CPU cores available: {cores}")

Great! We have 2 cores available. 

Another key aspect of our setup is the allocation of 64GB of memory. This provides ample resources for libraries like xgboost (which we will use for training) to efficiently process data and perform computations directly in memory, ensuring reliable performance during training.

***NOTE***: *In this module, we selected Dask as an example of a library well-suited for handling big data. However, with the current setup using only 2 cores, processing the massive datasets generated by both ALS and TLS remains challenging. For the data challenge, we encourage you to explore alternative libraries and frameworks that align better with your specific requirements. 
To do this effectively, you'll need to configure the appropriate environment within the NDP JupyterHub. We recommend reviewing the provided [documentation](https://nationaldataplatform.org/documentation/jupyter/bring-your-own-image/), which offers step-by-step instructions for building your own Docker image, including the one used in this demo.*

Now, lets build a simple classifier:

In [ ]:
import dask.array as da

# 1. First, we will transform our features into Dask arrays
x_coords = da.from_array(als_points.x, chunks=10000)
y_coords = da.from_array(als_points.y, chunks=10000)
z_coords = da.from_array(als_points.z, chunks=10000)
intensity = da.from_array(als_points.intensity, chunks=10000)
classification = da.from_array(als_points.classification, chunks=10000)  # This is the target variable

# 2. Now, we define sample size and generate random indices
sample_size = 2_000_000  # For this example, we will only use a sample of 2,000,000 points, otherwise training time would be more extensive.
total_size = x_coords.shape[0]
random_indices_np = np.random.choice(total_size, size=sample_size, replace=False)
random_indices = da.from_array(random_indices_np, chunks=10000).compute()

# 3. Now we convert Dask arrays to NumPy arrays for selected samples
x_sample = x_coords.compute()[random_indices]
y_sample = y_coords.compute()[random_indices]
z_sample = z_coords.compute()[random_indices]
intensity_sample = intensity.compute()[random_indices]
classification_sample = classification.compute()[random_indices]

# 4. We stack all features in a single array for easier processing
features = np.stack([x_sample, y_sample, z_sample, intensity_sample], axis=1)

One thing about this data, is the high unbalance between class 1 and the rest of the classes. 

In [ ]:
from collections import Counter

class_counts = Counter(als_points.classification)
total_points = sum(class_counts.values())

for cls, count in class_counts.items():
    percentage = (count / total_points) * 100
    print(f"Class {cls}: {percentage:.2f}%")

Class 1 represents around 73% of the total points. Therefore, we need to account for that in our training design.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# 5. We perform a stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    features, classification_sample, stratify=classification_sample, test_size=0.2, random_state=42
)

# 6. Then, we map non-contiguous class labels to contiguous ones - This is a required step for XGBoost
label_encoder = LabelEncoder()
y_train_mapped = label_encoder.fit_transform(y_train)
y_test_mapped = label_encoder.transform(y_test)

# 7. Now, we apply SMOTE to oversample minority classes, otherwise our model will struggle to learn
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train_mapped)

With our data prepared and the class imbalances accounted for, we can now proceed with the training process.

In [ ]:
# FYI This cell will take around 3 min to run
from xgboost import XGBClassifier

# 8. With the data ready, we initialize and train the XGBoost model
model = XGBClassifier(
    objective="multi:softmax",  # Multi-class classification
    num_class=len(np.unique(y_resampled)),  # Number of classes
    random_state=42,
    eval_metric="mlogloss"
)
model.fit(X_resampled, y_resampled)

# 9. After training, we predict on the test set
y_pred_mapped = model.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_mapped) # We return the original labels to our classes

Success! Our model is now trained and ready to be evaluated. We will evaluate using the accuracy score, a classification report and printing the confusion matrix.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 10. Now we have to evaluate the model
print(f"Accuracy Score: {accuracy_score(y_test, y_pred) * 100:.2f}%\n")

print("Classification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

The classifier demonstrates decent performance, achieving an accuracy of 71.50%. However, it exhibits low precision for labels 7 and 20, which correspond to noise points and ignored ground, respectively. This outcome is expected, as these labels are underrepresented in our dataset, making it more challenging for the model to learn and predict them accurately.

As we discussed earlier, a classifier like this can serve as a valuable tool to assist in the classification of points captured in the field. While this example focuses on simplicity, it’s worth noting that the classifier could be significantly improved by incorporating more advanced techniques such as feature engineering, ensemble methods, or deep learning.

For the purposes of this demo, however, we have prioritized a straightforward approach to illustrate the foundational concepts behind building and evaluating a point cloud classifier.

## Additional Resources

In this module, we’ve provided a brief introduction to ALS (Airborne Laser Scanning) and TLS (Terrestrial Laser Scanning) data. Your challenge will be to identify a meaningful use case for this data and apply it to develop more effective fire models.

To wrap up this module, we encourage you to dive deeper by exploring the following resources:

- https://www.usgs.gov/faqs/what-lidar-data-and-where-can-i-download-it
- https://oceanservice.noaa.gov/geodesy/lidar.html
- https://research.fs.usda.gov/srs/products/compasslive/measuring-forest-structure-lidar-and-finding-right-resolution
- https://sweri.eri.nau.edu/wp-content/uploads/2019/11/FactSheet_LiDAR_Oct2019_FINAL.pdf
- https://research.fs.usda.gov/treesearch/30652